In [1]:
__author__ = 'kjoseph'
from array import array

from deflection import *
from initialize_functions import *
from functions import *
import scipy.spatial
FUNDAMENTALS = ['ae','ap','aa','be','bp','ba','oe','op','oa']

EQUATION_FILE = "../data/sexes_avg_new.txt"


t,M = get_t_and_M_from_file(EQUATION_FILE, FUNDAMENTALS)
deflection_computation = get_coefficients(M,t)
H = np.dot(np.vstack((np.identity(9),-M)), np.hstack((np.identity(9),-M.transpose())))
S_a,g_a = compute_s_and_g(M,t,'a',EPA_LIST)
S_b,g_b = compute_s_and_g(M,t,'b',EPA_LIST)
S_o,g_o = compute_s_and_g(M,t,'o',EPA_LIST)
##TEST DEFLECTION

pm1 = ParameterStruct('t', 1,3,3,[0.556179039209, 1],[0.748935820403, 1],[0.608249328496,1],1,1)
pm2 = ParameterStruct('t', 1,3,3,[-0.42344161868, 1],[-0.659712135792, 1],[0.192399427834,1],1,1)
opt_v = get_optimal(M,t,H,S_b,g_b,['a','o'], pm1,pm2)

for v in [[-0.447532713413, -0.356447458267, 0.429316014051], [-0.069087468087,0.258912533522,0.0270347073674]]:
    print scipy.spatial.distance.cosine(opt_v,v)

deflection_computation = np.array(deflection_computation, dtype=np.object_)


vals_list = array('f',[2.45, 1.75, 0.29,1.85, 1.65, 0.30,1.49,.31,.75])
for x in compute_deflection_bad(vals_list,deflection_computation):
    print 'x: ', x

# np.random.shuffle(M)
# deflection_computation = get_coefficients(M, t)
# print 'shuffled'
# for x in compute_deflection_bad(vals_list,deflection_computation):
#     print x
#for fundamental_index in range(N_FUNDAMENTALS):
#    c0, c1, c2 = get_constants_for_fundamental(fundamental_index,deflection_computation,value_list)
#    print FUNDAMENTALS[fundamental_index], -c1/(2*c0), c1, c0

ImportError: No module named deflection

In [179]:
def get_t_and_M_from_file(eq_filename, fundamentals,spl_char= "\t"):
    M = []
    t = []
    equation_file = open(eq_filename)
    i = 0
    for line in equation_file:
        t.append(set())
        line_spl = [l.strip() for l in line.split(spl_char)]
        M.append([float(x) for x in line_spl[1:]])

        coeff = line_spl[0].replace("Z","")
        for j in range(len(coeff)):
            if coeff[j] == '1':
                t[i].add(fundamentals[j])
        i+=1

    equation_file.close()
    return t, np.array(M)

FUNDAMENTALS = ['ae','ap','aa','be','bp','ba','oe','op','oa']

t, M = get_t_and_M_from_file("../data/sexes_avg_new.txt",FUNDAMENTALS,"\t")

fund_eq = [[] for i in range(len(FUNDAMENTALS))]
for j in range(len(FUNDAMENTALS)):
    for i,coef in enumerate(t):
        coef = "*".join(coef)
        l = M[i,j]
        app_str = ""
        if l > 0:
            app_str = "+"
        if l == 0:
            continue
        elif coef != '':
            fund_eq[j].append(app_str +str(l)+"*"+coef)
        else:
            fund_eq[j].append(app_str+str(l))


FUND_EQ_STRS = ["".join(x) for x in fund_eq]
print FUND_EQ_STRS

['-0.155+0.41*ae+0.42*be-0.02*bp-0.1*ba+0.03*oe+0.06*op+0.05*be*ae+0.03*ae*op+0.12*be*oe-0.05*be*op-0.05*oe*bp+0.03*be*oe*ae-0.02*be*ae*op', '-0.1+0.56*ap+0.06*aa-0.105*be+0.44*bp+0.04*oe-0.05*ap*bp+0.01*be*oe', '+0.14+0.05*ae+0.705*aa-0.06*be+0.29*ba-0.06*aa*ba', '-0.14+0.11*ae+0.555*be-0.12*ba+0.05*op+0.11*be*oe-0.05*be*op-0.02*oe*bp+0.02*be*oe*ae', '+0.06+0.16*ap-0.15*be+0.685*bp+0.03*oe-0.015*op+0.01*be*ae+0.02*be*oe', '+0.17+0.02*ae-0.06*ap+0.3*aa+0.04*be+0.64*ba', '+0.025+0.11*be+0.61*oe-0.01*oa+0.03*be*ae+0.04*be*oe-0.03*oe*bp', '-0.395+0.15*be-0.11*bp-0.115*oe+0.66*op+0.07*oa+0.03*be*oe+0.03*be*op-0.05*bp*op', '-0.035+0.02*be+0.03*oe-0.05*op+0.745*oa']


In [176]:
from sympy.solvers import solve
from sympy import Symbol
from sympy import sympify
from sympy.polys import Poly
from math import sqrt 


identities = [1, 2]
sentiment_words = [1,4]
constraints = [SocialEventConstraint(actor=1, behavior=1, object=2),
               EqualityConstraint(identity=1, sentiment_word=1)]
               
equation_str = ''

for starter, list_of_terms in  [['i',identities], ['z',sentiment_words]]:
    for term in list_of_terms:
        for epa in ['e','p','a']:
            id = starter+str(term)+epa
            symbols[id] = Symbol(id)

eq_constr = [constraint.get_constraint_string() for constraint in constraints]
equation_str = "+".join(eq_constr) 
expr = sympify(equation_str)

dat = {"i1e": 1.39,
       "i1p": .88,
       "i1a": 0.96,
       "z1e": -1.92,
       "z1p": 1.00,
       "z1a": 1.62,
       "i2e": 1.49,
       "i2p": .31,
       "i2a": 0.75,
      }
substitutions = dat.items()
expr = expr.subs(substitutions).expand()
p = Poly(expr).coeffs()
print expr
print -p[1]/(2*p[0])

TypeError: get_constraint_string() takes exactly 2 arguments (1 given)

In [ ]:
class User:
    
    def __init__(self, n_identities, sentences):
        self.sentences = sentences
        self.sentences_for_identity = [list() * n_identities]
        for sent_it, sentence in enumerate(sentences):
            self.add_sentence(sentence,sent_it)
            
    def add_sentence(sentence,sent_it=None):
        self.sentences.append(sentence)
        if not sent_it:
            sent_it = len(sentence)
        for identity in sentence.identities_contained():
            self.sentences_for_identity[identity].append(sent_it)

In [ ]:
d = read_grouped_file

for g in d:
    construct_sentence
    add_to_user

In [155]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn

_wnl = WordNetLemmatizer()

TypeError: __init__() takes exactly 1 argument (2 given)

In [152]:
sent_dict = {}
for x in codecs.open("../../../../thesis/thesis_work/lcss_study/data/all_epa_terms.txt","r","utf8"):
    x_spl = x.split("\t")
    sent_dict[x_spl[0]] = [float(z) for z in x_spl[1:]]


IDENTITY_LIST_FN = "../../../../thesis/thesis_work/lcss_study/data/identities_for_study.txt"
identity_set = {x.strip().lower() for x in open(IDENTITY_LIST_FN)}
full_set_of_interesting_terms = identity_set|set(sent_dict.keys())

In [171]:
_wnl.lemmatize("stuck",wn.VERB)

u'stick'

In [174]:
sent_dict['deal with']

KeyError: 'deal with'

In [180]:
identity_set = {i : x.strip().lower() for i,x in enumerate(open(IDENTITY_LIST_FN))}

In [182]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load_word2vec_format("../../../../thesis/thesis_work/identity_extraction/python/gensim_model/glove_twitter_50_raw_model.txt.gz", binary=False)

In [185]:
'miracle' in sent_dict

True